In [1]:
import pandas as pd 
import numpy as np
import re

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
# list of stopwords like articles, preposition
stop = set(stopwords.words('english'))
from string import punctuation
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support

from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



In [2]:
#read the csv file
data = pd.read_csv('IMDB.csv')

In [3]:
data.shape

(1000, 12)

In [5]:
#data.Genre.value_counts()

In [6]:
mlb=MultiLabelBinarizer()
Y=mlb.fit_transform(data['Genre'])

In [8]:
print (Y.shape)
print (np.sum(Y, axis=0))

(1000, 31)
[895 120 417  81 403 513 255 148 127 141 138 195  20 691 493 479 814  81
 272 690 250 765 576 767  91 897 258 572 280 259 569]


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle

In [26]:
list_of_values = ['Title', 'Genre', 'Description']

data = data[list(list_of_values)]
data = shuffle(data)
X = data.drop('Genre', axis = 1)

In [16]:
#data.head()

In [27]:
# The min_df paramter makes sure we exclude words that only occur very rarely
# The default also is to exclude any words that occur in every movie description
vectorize=CountVectorizer(max_df=0.95, min_df=0.005)
X=vectorize.fit_transform(list(data['Description']))

In [28]:
X.shape

(1000, 812)

In [29]:
from sklearn.feature_extraction.text import TfidfTransformer

In [30]:
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X)
X_tfidf.shape

(1000, 812)

In [31]:
msk = np.random.rand(X_tfidf.shape[0]) < 0.8

In [32]:
X_train_tfidf=X_tfidf[msk]
X_test_tfidf=X_tfidf[~msk]
Y_train=Y[msk]
Y_test=Y[~msk]


The accuracy of the KNN is 0.0
             precision    recall  f1-score   support

          0       0.90      0.97      0.94       173
          1       0.00      0.00      0.00        16
          2       0.37      0.14      0.20        72
          3       0.00      0.00      0.00        14
          4       0.35      0.08      0.14        83
          5       0.56      0.37      0.44        92
          6       0.14      0.03      0.05        37
          7       0.00      0.00      0.00        31
          8       0.00      0.00      0.00        35
          9       0.00      0.00      0.00        40
         10       0.00      0.00      0.00        18
         11       0.00      0.00      0.00        35
         12       0.00      0.00      0.00         5
         13       0.70      0.76      0.73       129
         14       0.46      0.30      0.36        97
         15       0.47      0.16      0.24        95
         16       0.86      0.76      0.81       163
         17   

/Users/sanjay/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [33]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report


In [35]:
parameters = {'kernel':['linear'], 'C':[0.01, 0.1, 1.0]}
gridCV = GridSearchCV(SVC(class_weight='balanced'), parameters, scoring=make_scorer(f1_score, average='micro'))
classif = OneVsRestClassifier(gridCV)

classif.fit(X_train_tfidf, Y_train)

OneVsRestClassifier(estimator=GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),...refit=True, return_train_score=True,
       scoring=make_scorer(f1_score, average=micro), verbose=0),
          n_jobs=1)

In [65]:
from sklearn import metrics
predstfidf=classif.predict(X_test_tfidf)
print('The accuracy of the Logistic Regression is',metrics.accuracy_score(predstfidf,Y_test))
#print (classification_report(Y_test, predstfidf, target_names=data['Genre']))

The accuracy of the Logistic Regression is 0.0
                            precision    recall  f1-score   support

   Action,Adventure,Sci-Fi       0.87      0.81      0.84       189
  Adventure,Mystery,Sci-Fi       0.15      0.27      0.19        22
           Horror,Thriller       0.34      0.48      0.40        81
   Animation,Comedy,Family       0.07      1.00      0.13        15
  Action,Adventure,Fantasy       0.35      0.37      0.36        83
  Action,Adventure,Fantasy       0.56      0.52      0.54       122
        Comedy,Drama,Music       0.26      0.38      0.31        56
                    Comedy       0.16      1.00      0.28        35
Action,Adventure,Biography       0.18      0.23      0.20        30
   Adventure,Drama,Romance       1.00      0.05      0.10        38
  Adventure,Family,Fantasy       0.12      0.25      0.17        24
   Biography,Drama,History       0.19      1.00      0.32        41
   Action,Adventure,Sci-Fi       0.00      0.00      0.00         3


/Users/sanjay/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [64]:
from sklearn.naive_bayes import MultinomialNB
classifnb = OneVsRestClassifier(MultinomialNB())
classifnb.fit(X[msk].toarray(), Y_train)
predsnb=classifnb.predict(X[~msk].toarray())
print('The accuracy of the Logistic Regression is',metrics.accuracy_score(predsnb,Y_test))

The accuracy of the Logistic Regression is 0.00462962962963


In [56]:
predictionsnb=[]
for i in range(X_test_tfidf.shape[0]):
    pred_genres=[]
    movie_label_scores=predsnb[i]
    for j in range(20):
        #print j
        if movie_label_scores[j]!=0:
            genre=data['Genre'][j]
            pred_genres.append(genre)
    predictionsnb.append(pred_genres)

In [63]:
for i in range(X_test_tfidf.shape[0]):
    if i%50==0 and i!=0:
        print ('MOVIE: ',data['Title'][i],'\tGenre: ',data['Genre'][i], '\tPREDICTION: ',','.join(predictionsnb[i]))

MOVIE:  Star Wars: Episode VII - The Force Awakens 	Genre:  Action,Adventure,Fantasy 	PREDICTION:  Action,Adventure,Sci-Fi,Horror,Thriller,Action,Adventure,Fantasy,Animation,Adventure,Comedy,Action,Comedy,Drama,Animation,Adventure,Comedy,Biography,Drama,History,Drama,Mystery,Sci-Fi
MOVIE:  Legend 	Genre:  Biography,Crime,Drama 	PREDICTION:  Action,Adventure,Sci-Fi,Action,Adventure,Fantasy,Comedy,Animation,Adventure,Comedy,Biography,Drama,History
MOVIE:  The BFG 	Genre:  Adventure,Family,Fantasy 	PREDICTION:  Action,Adventure,Sci-Fi,Horror,Thriller,Action,Adventure,Fantasy,Animation,Adventure,Comedy,Biography,Drama,History,Drama,Mystery,Sci-Fi
MOVIE:  Edge of Tomorrow 	Genre:  Action,Adventure,Sci-Fi 	PREDICTION:  Action,Adventure,Sci-Fi,Horror,Thriller,Action,Adventure,Fantasy,Animation,Adventure,Comedy,Animation,Adventure,Comedy,Biography,Drama,History,Drama,Mystery,Sci-Fi


In [66]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model_textual = Sequential([
    Dense(300, input_shape=(300,)),
    Activation('relu'),
    Dense(20),
    Activation('softmax'),
])

model_textual.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Using TensorFlow backend.


In [68]:
X.shape

(1000, 812)

In [72]:
mask_text = np.random.rand(X.shape[0]) < 0.8

In [73]:
X_train=X[mask_text]
Y_train=Y[mask_text]
X_test=X[~mask_text]
Y_test=Y[~mask_text]

In [81]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model_textual = Sequential([
    Dense(812, input_shape=(812,)),
    Activation('relu'),
    Dense(31),
    Activation('softmax'),
])

model_textual.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [77]:
Y_train

array([[1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 1, ..., 1, 1, 1],
       [1, 0, 0, ..., 0, 0, 1]])

In [87]:
#model_textual.fit(X_train, Y_train)